# Conversations versus context

In this notebook we will learn how to

1. Why you are not having a real conversation with a LLM. Inference uses the full context (conversation history) as a prompt.
3. How to include conversation history in prompts to local LLMs using `ollama-python`

> Ollama python package can be installed from [PyPI](https://pypi.org/project/ollama/). I have provided a [conda environment](./environment.yml) file that will install it for you.

## Imports

In [1]:
import ollama
from ollama import chat

## Open models installed on local machine

In [2]:
def installed_models():
    '''
    Iterate through ollama models and return names as list
    '''
    return [md.model for md in ollama.list().models]

In [3]:
local_models = installed_models()
local_models

['phi4:latest', 'deepseek-r1:1.5b', 'deepseek-r1:7b']

## Prompt DeepSeek-R1:1.5B

First let's attempt to have a conversation with DeepSeek-R1 1.5 Billion parameter model. We'll ask it to code a trivial function in python and then we will continue the conversation and ask for the code to be translated to Julia.

In [4]:
prompt_1 = """
Code a function in python that converts fahrenheit to 
celsius
"""

In [5]:
# use 1.5b parameter model
response_1 = chat(
    model=local_models[1],
    messages=[{'role': 'user', 'content': prompt_1}],
    stream=False,
)

In [6]:
print(response1.message.content)

<think>
Okay, I need to write a Python function that converts Fahrenheit to Celsius. Let me think about how to approach this.

First, I remember the formula for converting Fahrenheit (F) to Celsius (C). It's C = (F - 92) * 5/9 or wait, no, let me check that again. Oh right, it should be (F - 32) multiplied by 5/9. So the correct formula is C = (F - 32) * (5/9).

So my function will take a Fahrenheit temperature as input and return the equivalent Celsius.

Let's outline the steps:

1. Define a function named fahrenheit_to_celsius that takes one argument, F.
2. Inside the function, apply the formula: subtract 32 from F, multiply by 5/9.
3. Return the result.

I should make sure to handle any potential data types correctly. Since F is presumably a number (int or float), there shouldn't be issues here.

Testing some examples:

For example, if F is 32, then C should be 0. Let me plug into the formula: (32 -32)*5/9=0. Correct.

Another test case: 212°F should convert to 100°C. Using the form

In [7]:
prompt_2 = """
Thank you! Now convert the function you just coded from Python to C#.
"""

In [8]:
response_2 = chat(
    model=local_models[1],
    messages=[{'role': 'user', 'content': prompt_2}],
    stream=False,
)

In [9]:
print(response_2.message.content)

<think>
Okay, I need to figure out how to convert a Python function into C#. Let's start by understanding what the original Python code does. The user mentioned they have a function but didn't specify it, so I'll make some assumptions based on common tasks.

First, if the function is about converting text to ASCII values, that's straightforward. In Python, I can use the `ord()` function for lowercase letters and `chr()` for uppercase. But in C#, we don't have these built-in functions by default, so I'll need a custom method.

So, I'll write a method called `TextToASCII` that takes a string and returns another object (probably an array of integers) containing each character's ASCII value. For lowercase letters, it will be 'a'-'z', which correspond to 97-122, and for uppercase, it's 'A'-'Z', from 65 to 90.

I should make sure the method handles each character correctly. If a character isn't in the ASCII range (32 to 126), I might return null or handle it differently, but perhaps just lea

## Conversation history

In [10]:
def format_message(history: list, role: str, content: str):
    """
    Format the chat history

    Parameters:
    ----------
    history: list
        List containing chat history.

    role: str
        'user' or 'assistant' 

    content: str
        content to add to chat history
    """
    prompt = {
        'role': role,
        'content': content
    }
    history.append(prompt)
    return history

In [11]:
messages = [
  {
    'role': 'user',
    'content': prompt_1,
  },
  {
    'role': 'assistant',
    'content': response1.message.content,
  },
]

In [12]:
messages = format_message(messages, 'assistant', prompt_2)

In [13]:
print(messages[2])

{'role': 'assistant', 'content': '\nThank you! Now convert the function you just coded from Python to C#.\n'}


In [14]:
response_2 = chat(
    model=local_models[1],
    messages=messages,
    stream=False,
)

In [15]:
print(response_2.message.content)

The final answer is stored in a variable c which is declared with the type int. This will return an integer value representing the temperature in Celsius.

Here's how we can modify our code for C#:

```csharp
using System;

class Program 
{
    public static int FahrenheitToCelsius(int fahrenheit)
    {
        return (fahrenheit - 32) * 5 / 9;
    }
}
```

This function will take an integer as input, perform the temperature conversion calculation, and return an integer result.


## Pass the conversation to a different model.

We will try DeepSeek-R 7 billion parameter model.

We can pass the conversation history we had with the 1.5 billion model to a different instance of the model and inference will be successful.

In [27]:
response_2_7b = chat(
    model=local_models[2],
    messages=messages,
    stream=False,
)

In [29]:
print(response_2_7b.message.content)

Alright, I need to convert the Python function into a C# function. Let's see... The original Python function takes a Fahrenheit temperature and returns the Celsius equivalent using the formula (F - 32) * 5/9.

First, in C#, functions are defined differently than in Python. They take parameters explicitly, so I'll need to create a method that accepts a double for Fahrenheit and returns a double.

I should make sure to use the correct formula: Celsius = (Fahrenheit - 32) * 5 / 9.

Let me write the C# code step by step:
- Create a public static method named ConvertFahrenheitToCelsius.
- The parameter is of type double, called Fahrenheit.
- Inside the method, compute Celsius as (Fahrenheit - 32.0) * 5.0 / 9.0.
- Return the computed Celsius value.

Testing this with some examples: For F = 32, C should be 0. Using the formula, (32 - 32)*5/9 is indeed 0. Another example, F = 212 converts to 100°C because (212 - 32) * 5 / 9 equals 180 * 5 /9 which is 100.

I should also consider edge cases lik